In [1]:
import math

In [2]:
import gurobipy as gp

In [3]:
from gurobipy import GRB

**Setup**

In [43]:
height = 10
width = 10
spaces = height * width

In [44]:
#set snakes
inputSnakePairs = [[3, 37], [6, 16], [15, 9], [49, 12], [14, 32],
  [27, 56], [61, 22], [42, 17], [88, 36], [39, 44],
  [58, 45], [75, 47], [94, 64], [97, 65], [69, 87],
  [79, 98], [41, 85], [89, 91]]
snakePairs = []
for [head, tail] in inputSnakePairs:
    if head < spaces - 1 and tail < spaces - 1 and abs(head-tail) >= 2:
        snakePairs.append([head-1, tail-1])
snakes = len(snakePairs)

In [45]:
model = gp.Model('SnakesAndLadders')

In [46]:
vars = model.addVars(height, width, spaces, vtype=GRB.BINARY, name = 'VAR')

In [47]:
# dist[s, d] = distance of snake s in dimension d (d = 0: vertical, d = 1: horizontal) 
row = model.addVars(spaces, vtype=GRB.INTEGER, lb=0, ub=height-1, name = 'ROW')
col = model.addVars(spaces, vtype=GRB.INTEGER, lb=0, ub=width-1, name = 'COL')
offset = model.addVars(snakes, 2, vtype=GRB.INTEGER, lb = -(width-1), ub = width-1, name = 'OFF')
dist = model.addVars(snakes, 2, vtype=GRB.INTEGER, lb = 0, ub=width-1, name = 'DST')

In [48]:
# display solution
def getSpace(i, j):
    result = -1
    val = model.getAttr('X', vars)
    for v in range(spaces):
        if val[i, j, v] > 0.5:
            result = v;
    return result

def getIJ(v):
    result = [-1, -1]
    val = model.getAttr('X', vars)
    for i in range(height):
        for j in range(width):
            if val[i, j, v] > 0.5:
                result = [i, j]
    return result
    
def displayBoard():
    vals = model.getAttr('X', vars)
    for i in range(height):
        output_string = ''
        for j in range(width):
            for v in range(spaces):
                if vals[i, j, v] > 0.5:
                    output_string += str(getSpace(i, j)+1).ljust(4)
        print(output_string)

**Board, path constraints**

In [49]:
# Fix variables: even and odd spaces are arranged as checkerboards
for i in range(height):
    for j in range(width):
        for v in range(spaces):
            if (i+j+v) % 2 == 1 :
                vars[i, j, v].UB = 0

In [50]:
model.addConstrs((vars.sum(i, j, '*') == 1
                 for i in range(height)
                 for j in range(width)), name='V')
model.update()

In [51]:
model.addConstrs((vars.sum('*', '*', v) == 1
                 for v in range(spaces)), name='S')
model.update()

In [52]:
def setNeighbors(v):
    V = v + 1
    for i in range(1, height-1):
        model.addConstr(vars[i, 0, v] <= vars[i-1, 0, V] + vars[i, 1, V] + vars[i+1, 0, V], name='left')
        model.addConstr(vars[i, width-1, v] <= vars[i-1, width-1, V] + vars[i, width-2, V] + vars[i+1, width-1, V], name='right')
    for j in range(1, width-1):
        model.addConstr(vars[0, j, v] <= vars[0, j-1, V] + vars[1, j, V] + vars[0, j+1, V], name='top')
        model.addConstr(vars[height-1, j, v] <= vars[height-1, j-1, V] + vars[height-2, j, V] + vars[height-1, j+1, V], name='bottom')
    for i in range(1, height-1):
        for j in range(1, width-1):
            model.addConstr(vars[i, j, v] <= vars[i-1, j, V] + vars[i, j-1, V] + vars[i, j+1, V] + vars[i+1, j, V], name='middle')
    model.addConstr(vars[0, 0, v] <= vars[1, 0, V] + vars[0, 1, V], name='tl')
    model.addConstr(vars[0, width-1, v] <= vars[1, width-1, V] + vars[0, width-2, V], name='tr')
    model.addConstr(vars[height-1, 0, v] <= vars[height-2, 0, V] + vars[height-1, 1, V], name='bl')
    model.addConstr(vars[height-1, width-1, v] <= vars[height-2, width-1, V] + vars[height-1, width-2, V], name='br')

# set ordered path
for v in range(spaces-1):
    setNeighbors(v)

**Snake distance variables**

In [53]:
for v in range(spaces):
    model.addConstr(row[v] == sum(vars[i, j, v] * i for i in range(height) for j in range(width)), name="ROW"+str(v)) 
    model.addConstr(col[v] == sum(vars[i, j, v] * j for i in range(height) for j in range(width)), name="COL"+str(v))
    
for s in range(snakes):
    [head, tail] = snakePairs[s]
    model.addConstr(offset[s, 0] == row[head] - row[tail], name="OFFS"+str(s)+",0")
    model.addConstr(offset[s, 1] == col[head] - col[tail], name="OFFS"+str(s)+",1")
    model.addGenConstrAbs(dist[s, 0], offset[s, 0], name="DIST"+str(s)+",0")
    model.addGenConstrAbs(dist[s, 1], offset[s, 1], name="DIST"+str(s)+",1")

In [54]:
model.update()
model.optimize()    
displayBoard()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 10336 rows, 10272 columns and 83848 nonzeros
Model fingerprint: 0x169b8e07
Model has 36 general constraints
Variable types: 0 continuous, 10272 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5186 rows and 5272 columns
Presolve time: 0.11s
Presolved: 5150 rows, 5000 columns, 32770 nonzeros
Variable types: 0 continuous, 5000 integer (5000 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.14 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
1   2   3   4   5   6   7   8   9   10  
20  19  18  17  16  15  14  13  12  11  
21  24  25  26  27  28  29  30  31  32  
22 

In [55]:
model.setObjective(dist.sum(), GRB.MINIMIZE)

In [42]:
model.update()
model.optimize()

displayBoard()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 2560 rows, 2523 columns and 19468 nonzeros
Model fingerprint: 0x81ab1409
Model has 12 general constraints
Variable types: 0 continuous, 2523 integer (2401 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 34

Presolve removed 1238 rows and 1286 columns
Presolve time: 0.03s
Presolved: 1322 rows, 1237 columns, 8210 nonzeros
Variable types: 0 continuous, 1237 integer (1201 binary)

Root relaxation: objective 0.000000e+00, 1294 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  334   34.00000    0.00000   100%     -    0s
     0     0    0.66667    0  476   34.00000    0.66667  98.0%     -  

**Scratch space**

In [ ]:
val = el.getAttr('X', vars)